# 1) Loading models

In [1]:
from transformers import pipeline

In [136]:
import torch
device = 0 if torch.cuda.is_available() else -1
device

-1

In [133]:
models = {
    "HerBERT": "allegro/herbert-base-cased",
    "Facebook": "FacebookAI/xlm-roberta-base",
    "Papuga": "flax-community/papuGaPT2"
}

In [137]:
pipelines = {}

for model_name, model_path in models.items():
  if model_name == "Papuga":
    pipelines[model_name] = pipeline(task="text-generation", model=model_path)
  else:
    pipelines[model_name] = pipeline(task="fill-mask", model=model_path)

Device set to use cpu
Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Device set to use cpu


In [140]:
text = "Geralt z Rivii jest <mask> wiedźminem ze szkoły wilka."
preds = pipelines['HerBERT'](text, top_k=1)
print(preds)
preds = pipelines['Facebook'](text, top_k=1)
print(preds)
text = "Geralt z Rivii jest wiedźminem ze szkoły wilka. I jest on "
preds = pipelines['Papuga'](text, top_k=1)
print(preds)

[{'score': 0.08581207692623138, 'token': 4541, 'token_str': 'pierwszym', 'sequence': 'Geralt z Rivii jest pierwszym wiedźminem ze szkoły wilka .'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'score': 0.10300716012716293, 'token': 96446, 'token_str': 'pierwszym', 'sequence': 'Geralt z Rivii jest pierwszym wiedźminem ze szkoły wilka.'}]
[{'generated_text': 'Geralt z Rivii jest wiedźminem ze szkoły wilka. I jest on owiany tajemnicą.\nW świecie, w którym magia jest wszechobecna, a magia jest wszechobecna, a magia jest wszechobecna, a magia jest wszechobec'}]


# 2) Nominativus

In [157]:
test_cases_nominativus = {
  "Mianownik": [
    "Kraków to moje miasto <mask>, w którym się urodziłem.",
    "Wieliczka to <mask> na skale soli."
  ],
  "Dopełniacz": [
    "Widziałem <mask> cień, latającego po niebie.",
    "W Japoni widziałem super szybki <mask>."
  ],
  "Celownik": [
    "Przyglądam się temu <mask> na Antarktydzie.",
    "Nauczyciel daje uwagi swoim <mask>"
  ],
  "Biernik": [
    "Wczoraj oglądałem <mask> o imieniu Krzysztof w telewizji.",
    "Ukradli cenny <mask> z muzeum."
  ],
  "Narzędznik": [
    "Jadę nowym wodorowym <mask> do kościoła.",
    "Idę do teatru w swojej nowej <mask>"
  ],
  "Miejscownik": [
    "Paież często mówił o <mask> z Wadowic.",
    "Przeglądając zdjęcia z <mask> wracają wspomnienia."
  ],
  "Wołacz": [
    "<mask> ,chodź tutaj!",
    "<mask> Ty mały kundlu!"
  ]
}

In [158]:
test_cases_nominativus_papuga = {
    "Mianownik": [
        "Kraków to moje miasto, w którym się urodziłem. To miasto jest ",
        "Wieliczka to miasto zbudowane obok kopalni  "
    ],
    "Dopełniacz": [
        "Widziałem cień czegoś niezwykłego na niebie. Był to ",
        "W Japonii widziałem bardzo szybki  "
    ],
    "Celownik": [
        "Przyglądam się temu na Antarktydzie. To jest ",
        "Nauczyciel daje uwagi swoim uczniom, ale także "
    ],
    "Biernik": [
        "Wczoraj oglądałem w telewizji program o nazwie ",
        "Ukradli cenny obraz z muzeum. Ten obraz przedstawiał "
    ],
    "Narzędznik": [
        "Jadę nowym samochodem wodorowym, a ten samochód jest naprawdę ",
        "Idę do teatru w swojej nowej sukni, która wygląda "
    ],
    "Miejscownik": [
        "Papież często mówił o swoim rodzinnym mieście. To było ",
        "Przeglądając zdjęcia z wakacji, wspomnienia wracają o miejscach takich jak "
    ],
    "Wołacz": [
        "Hej, mój przyjacielu! Czy możesz przyjść tutaj, ",
        "Ty, mały kundlu, przestań szczekać i słuchaj, "
    ]
}


In [163]:
case_results_nominativus = {}

for model_name, model in pipelines.items():
  case_results_nominativus[model_name] = {
    "Mianownik": [],
    "Dopełniacz": [],
    "Celownik": [],
    "Biernik": [],
    "Narzędznik": [],
    "Miejscownik": [],
    "Wołacz": []
  }
case_results_nominativus

{'HerBERT': {'Mianownik': [],
  'Dopełniacz': [],
  'Celownik': [],
  'Biernik': [],
  'Narzędznik': [],
  'Miejscownik': [],
  'Wołacz': []},
 'Facebook': {'Mianownik': [],
  'Dopełniacz': [],
  'Celownik': [],
  'Biernik': [],
  'Narzędznik': [],
  'Miejscownik': [],
  'Wołacz': []},
 'Papuga': {'Mianownik': [],
  'Dopełniacz': [],
  'Celownik': [],
  'Biernik': [],
  'Narzędznik': [],
  'Miejscownik': [],
  'Wołacz': []}}

In [164]:
for model_name, model in pipelines.items():
  if model_name == "Papuga":
    break
  for case, sents in test_cases_nominativus.items():
    case_result = []
    for sent in sents:
      preds = model(sent, top_k=1)
      case_result.append((preds[0]['score'],preds[0]['token_str'],preds[0]['sequence']))
    case_results_nominativus[model_name][case] = case_result

In [173]:
model_name = pipelines['Papuga']
for case, sents in test_cases_nominativus_papuga.items():
  case_result = []
  for sent in sents:
    preds = model(sent, top_k=1)
    case_result.append((sent,preds[0]['generated_text']))
  case_results_nominativus['Papuga'][case] = case_result

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [174]:
YELLOW = '\033[93m'
GREEN = '\033[92m'
RED = '\033[91m'
RESET = '\033[0m'

In [176]:
for model_name, data in case_results_nominativus.items():
  print(f"Results for model: {GREEN}{model_name}{RESET}")
  for case, list_ in data.items():
    print(f"Results for nominativus: {RED}{case}{RESET}")
    for item_ in list_:
      if model_name == 'Papuga':
        print(f"To fill sentence {YELLOW}{item_[0]}{RESET} full sentence {YELLOW}{item_[1]}{RESET}")
      else:
        print(f"Score {item_[0]}, filled word {YELLOW}{item_[1]}{RESET} full sentence {YELLOW}{item_[2]}{RESET}")
    print("----------------")

Results for model: HerBERT
Results for nominativus: Mianownik
Score 0.78911954164505, filled word rodzinne full sentence Kraków to moje miasto rodzinne , w którym się urodziłem .
Score 0.23422826826572418, filled word miasto full sentence Wieliczka to miasto na skale soli .
----------------
Results for nominativus: Dopełniacz
Score 0.7132082581520081, filled word jego full sentence Widziałem jego cień , latającego po niebie .
Score 0.1343093365430832, filled word samolot full sentence W Japoni widziałem super szybki samolot .
----------------
Results for nominativus: Celownik
Score 0.1086496114730835, filled word miejscu full sentence Przyglądam się temu miejscu na Antarktydzie .
Score 0.716489851474762, filled word uczniom full sentence Nauczyciel daje uwagi swoim uczniom
----------------
Results for nominativus: Biernik
Score 0.29405054450035095, filled word człowieka full sentence Wczoraj oglądałem człowieka o imieniu Krzysztof w telewizji .
Score 0.4107305109500885, filled word zab

# 3) Long range relationship

In [184]:
def test_model(test_cases, test_cases_papuga):
  case_results = {}

  for model_name, model in pipelines.items():
    case_results[model_name] = []
  case_results

  for model_name, model in pipelines.items():
    case_result = []
    if model_name == 'Papuga': break
    for sent in test_cases:
      preds = model(sent, top_k=1)
      case_result.append((preds[0]['score'],preds[0]['token_str'],preds[0]['sequence']))
    case_results[model_name] = case_result

  model = pipelines['Papuga']
  for sent in test_cases_papuga:
    preds = model(sent, top_k=1)
    case_result.append((sent,preds[0]['generated_text']))
  case_results['Papuga'] = case_result

  for model_name, data in case_results.items():
    print(f"Results for model: {RED}{model_name}{RESET}")
    for item_ in data:
      if model_name == 'Papuga':
        print(f"To fill sentence {YELLOW}{item_[0]}{RESET}")
        print(f"full sentence {YELLOW}{item_[1]}{RESET}")
      else:
        print(f"Score {item_[0]}, filled word {YELLOW}{item_[1]}{RESET} full sentence {YELLOW}{item_[2]}{RESET}")
    print("----------------")

In [185]:
test_case_long_relationship = [
  "Bartek często chodzi na basen, to <mask> ulubione miejsce w mieście",
  "Gojo Satoru to mistrz sztuk jujutsu, to właśnie <mask> jest najpotężniejszym zaklinaczem w Japonii.",
  "On jest nienormalny zjada już kolejnego kebababa, ten <mask> to wariat!",
  "Królowa Jadwiga to pierwsza kobieta która została żeńskim <mask>",
  "Jan Paweł drugi to najepszy papież jaki był, bez <mask> Polska nie odzyskała by niepodległości"
]

test_case_long_relationship_papuga = [
  "Bartek często chodzi na basen, to miejsce jest jego",
  "Gojo Satoru to mistrz sztuk jujutsu. Zatem najpoteżniejszym zaklinaczem jujutsu jest ",
  "On jest nienormalny zjada już kolejnego kebababa, ten",
  "Królowa Jadwiga to pierwsza kobieta która została żeńskim",
  "Jan Paweł drugi to najepszy papież jaki był, Polska by nieodzyskała niepodległości gdyby nie"
]

In [186]:
test_model(test_case_long_relationship,test_case_long_relationship_papuga)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Results for model: HerBERT
Score 0.9546632766723633, filled word jego full sentence Bartek często chodzi na basen , to jego ulubione miejsce w mieście
Score 0.9092267751693726, filled word on full sentence Gojo Satoru to mistrz sztuk jujutsu , to właśnie on jest najpotężniejszym zaklinaczem w Japonii .
Score 0.27523764967918396, filled word facet full sentence On jest nienormalny zjada już kolejnego kebababa , ten facet to wariat !
Score 0.27282944321632385, filled word królem full sentence Królowa Jadwiga to pierwsza kobieta która została żeńskim królem
Score 0.8865628242492676, filled word niego full sentence Jan Paweł drugi to najepszy papież jaki był , bez niego Polska nie odzyskała by niepodległości
----------------
Results for model: Facebook
Score 0.8309106230735779, filled word jego full sentence Bartek często chodzi na basen, to jego ulubione miejsce w mieście
Score 0.8941888213157654, filled word on full sentence Gojo Satoru to mistrz sztuk jujutsu, to właśnie on jest najpotę

# 4) World knowladge

In [187]:
test_case_word_knowladge = [
  "Na wiosnę <mask> mają bardzo dużo pąków, natomiast jesiennią znacząco jest ich mniej.",
  "Ramen to danie pochodzące z <mask>",
  "Wilki, polując w <mask>, potrafią pokonać większe zwierzę, takie jak żubr",
  "Luty to <mask> miesiąc w roku",
  "Wawel to zamek, który możemy zobaczyć w <mask>"
]

test_case_word_knowladge_papuga = [
  "Na wiosne mają bardzo dużo pąków, natomiast jesiennią znacząco jest ich mniej. Chodzi mi o ",
  "Ramen to danie pochodzące z",
  "Wilki, polując potrafią pokonać większe zwierzę, takie jak żubr. Wszystko dlatego że polują w",
  "Luty to miesiąc w roku, który jest",
  "Wawel to zamek, który możemy zobaczyć w"
]

In [188]:
test_model(test_case_word_knowladge, test_case_word_knowladge_papuga)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Results for model: HerBERT
Score 0.34332987666130066, filled word kwiaty full sentence Na wiosnę kwiaty mają bardzo dużo pąków , natomiast jesiennią znacząco jest ich mniej .
Score 0.6201823949813843, filled word … full sentence Ramen to danie pochodzące z …
Score 0.4919811487197876, filled word lesie full sentence Wilki , polując w lesie , potrafią pokonać większe zwierzę , takie jak żubr
Score 0.16590219736099243, filled word ostatni full sentence Luty to ostatni miesiąc w roku
Score 0.15379521250724792, filled word … full sentence Wawel to zamek , który możemy zobaczyć w …
----------------
Results for model: Facebook
Score 0.25584104657173157, filled word kwiaty full sentence Na wiosnę kwiaty mają bardzo dużo pąków, natomiast jesiennią znacząco jest ich mniej.
Score 0.2061801254749298, filled word : full sentence Ramen to danie pochodzące z:
Score 0.06258589774370193, filled word człowieka full sentence Wilki, polując w człowieka , potrafią pokonać większe zwierzę, takie jak żubr
Sc

# 5) zero-shot learning

In [189]:
test_case_zero_shot = [
  "'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta ma jest zdecydowanie <mask>",
  "'Ta manga wymiata. Nie mogę przestać jej czytać!' Wypowiedź ta jest zdecydowanie <mask>",
  "'Ta manga wymiata. Nie mogę przestać jej czytać!' To zdanie jest nacechowane <mask>",
  "'Arnold Schwarzenegger to najlepszy aktor filmów akcji, a terminator judgment day to jego najepszy film.' Wypowiedź ta jest zdecydowanie <mask>",
  "'Arnold Schwarzenegger to najlepszy aktor filmów akcji, a terminator judgment day to jego najepszy film.' To zdanie jest nacechowane <mask>",
  "'Przepełnione autobusy to nieprzyjemne doświadczenie. Jazda w nich to istna męczarnia.' Wypowiedź ta jest zdecydowanie <mask>",
  "'Przepełnione autobusy to nieprzyjemne doświadczenie. Jazda w nich to istna męczarnia.' To zdanie jest nacechowane <mask>",
  "'Kremówki są takie słodkie! Ta słodycz sprawia mi wiele radości!' Wypowiedź ta jest zdecydowanie <mask>",
  "'Kremówki są takie słodkie! Ta słodycz sprawia mi wiele radości!' To zdanie jest nacechowane <mask>",
  "'W Starbucksie wypiłem najgorsza kawę w życiu. To był istny ściek.' Wypowiedź ta jest zdecydowanie <mask>",
  "'W Starbucksie wypiłem najgorsza kawę w życiu. To był istny ściek.' To zdanie jest nacechowane <mask>"
]

In [190]:
test_case_zero_shot_papuga = [
  "'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta ma jest zdecydowanie",
  "'Ta manga wymiata. Nie mogę przestać jej czytać!' Wypowiedź ta jest zdecydowanie",
  "'Ta manga wymiata. Nie mogę przestać jej czytać!' To zdanie jest nacechowane",
  "'Arnold Schwarzenegger to najlepszy aktor filmów akcji, a terminator judgment day to jego najepszy film.' Wypowiedź ta jest zdecydowanie",
  "'Arnold Schwarzenegger to najlepszy aktor filmów akcji, a terminator judgment day to jego najepszy film.' To zdanie jest nacechowane",
  "'Przepełnione autobusy to nieprzyjemne doświadczenie. Jazda w nich to istna męczarnia.' Wypowiedź ta jest zdecydowanie",
  "'Przepełnione autobusy to nieprzyjemne doświadczenie. Jazda w nich to istna męczarnia.' To zdanie jest nacechowane",
  "'Kremówki są takie słodkie! Ta słodycz sprawia mi wiele radości!' Wypowiedź ta jest zdecydowanie",
  "'Kremówki są takie słodkie! Ta słodycz sprawia mi wiele radości!' To zdanie jest nacechowane",
  "'W Starbucksie wypiłem najgorsza kawę w życiu. To był istny ściek.' Wypowiedź ta jest zdecydowanie",
  "'W Starbucksie wypiłem najgorsza kawę w życiu. To był istny ściek.' To zdanie jest nacechowane"
]

In [191]:
test_model(test_case_zero_shot,test_case_zero_shot_papuga)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Results for model: HerBERT
Score 0.13610470294952393, filled word . full sentence ' Ten film to był kiler . Nie mogłem się oderwać od ekranu . ' Wypowiedź ta ma jest zdecydowanie .
Score 0.12250640988349915, filled word . full sentence ' Ta manga wymiata . Nie mogę przestać jej czytać ! ' Wypowiedź ta jest zdecydowanie .
Score 0.5622760653495789, filled word . full sentence ' Ta manga wymiata . Nie mogę przestać jej czytać ! ' To zdanie jest nacechowane .
Score 0.19238553941249847, filled word lepsza full sentence ' Arnold Schwarzenegger to najlepszy aktor filmów akcji , a terminator judgment day to jego najepszy film . ' Wypowiedź ta jest zdecydowanie lepsza
Score 0.4942215383052826, filled word : full sentence ' Arnold Schwarzenegger to najlepszy aktor filmów akcji , a terminator judgment day to jego najepszy film . ' To zdanie jest nacechowane :
Score 0.16401365399360657, filled word prawdziwa full sentence ' Przepełnione autobusy to nieprzyjemne doświadczenie . Jazda w nich to istn

# Answering questions

#### Which of the models produced the best results?

The herbert model produced overall the best results. The only field where this model were worse than one of others to be exact Papuga is a World Knowladge case. Herbert was not albel to recognize Feburary as the shortest month and in cases where the last word should be last he produced `...`

#### Was any of the models able to capture Polish grammar?

More or less, papuga produced a lot of additional sentences that does not make any sense for example `To fill sentence Ty, mały kundlu, przestań szczekać i słuchaj,  full sentence Ty, mały kundlu, przestań szczekać i słuchaj, łkaj, bo nie ma już nic do stracenia.
 Nie, nie, nie, nie, nie, nie, nie, nie, nie, nie,` where papuga added a lot of `nie`. Looking at Facebook berta model it had a few hickups (listed in last question) but overall he did not bad.

#### Was any of the models able to capture long-distant relationships between the words?

Yes indeed HerBERT was excelent there, produced all words that filled the long sentences which referes to people or things from before. It is important to acknowladge papuga model here. Papuga in sentence `On jest nienormalny zjada już kolejnego kebababa, ten z kolei jest nienormalny.
A ja mam pytanie do tych co nie mają dzieci. Czy ktoś z was ma dzieci z zespołem Downa? Czy to jest normalne?` Papuga added `A ja mam pytanie do tych co nie mają dzieci. Czy ktoś z was ma dzieci z zespołem Downa? Czy to jest normalne?` which is suprising i mean where is the connection beetwen ill children and kebab?
<br>
In the other case with `Jadwiga` sentence `Królowa Jadwiga to pierwsza kobieta która została żeńskim` papuga and herbert were able to capture the complicated words `żeński król`

#### Was any of the models able to capture world knowledge?

Only one did pretty good papuga it is. HerBERT filled only first sentence correctly while Facebook did three well. In the second sentence `Ramen to danie pochodzące z` both fill-mask model were not able to fill the sentence while papuga was able to correctly identify that Ramen comes from Japan although some say that it first originated from China.

#### Was any of the models good at doing zero-shot classification?

None of the ones with filled mask, they always added only fullstop. On the other hand papuga did pretty well where the esntence to fill wa `To zdanie nacechowane jest` he did mention `bardzo emocjonalnie` or `negatywnie` but again it added also a lot of bullcrap.

#### What are the most striking errors made by the models?

Words or sentences add by model are listed below

1) Herbert
 - Luty to `ostatni` miesiąc w roku

2) Facebook berta
 - Wieliczka to`fu` na skale soli.
 - Idę do teatru w swojej nowej `wersji`
 - Królowa Jadwiga to pierwsza kobieta która została żeńskim `dzieckiem`
 - On jest nienormalny zjada już kolejnego kebababa, ten `jest` to wariat!
 - Wilki, polując w `człowieka` , potrafią pokonać większe zwierzę, takie jak żubr

3) Papuga
 - Idę do teatru w swojej nowej sukni, która wygląda `owiana tajemnicą. Nie, nie, nie. Nie, nie. Nie, nie. Nie, nie. Nie, nie. Nie, nie. Nie, nie. Nie`,
 - W Japonii widziałem bardzo szybki   `na 1,5 km, ale to było w połowie drogi do domu. W Japonii jest bardzo dużo miejsc, gdzie można się zatrzymać, ale nie ma ta zbyt wielu miejsc do siedzenia. W Japonii jest bardzo dużo`

